# Imports y config

In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
os.chdir('C:\\Users\\Usuario\\TFG\\digipanca\\')

# Dice Loss

In [47]:
from monai.losses import DiceLoss as MONAIDiceLoss
from src.losses import MulticlassDiceLoss

In [48]:
# ======================
# Datos de prueba corregidos
# ======================

batch_size = 4
n_classes = 5
height, width = 512, 512

# Crear tensores aleatorios de logits (NO etiquetas discretas)
y_pred = torch.randn(batch_size, n_classes, height, width)  # Logits de la red
y_true = torch.randint(0, n_classes, (batch_size, height, width))  # Ground truth en etiquetas

# Instancias de las pérdidas
my_dice_loss = MulticlassDiceLoss(ignore_background=True)
monai_dice_loss = MONAIDiceLoss(include_background=False, reduction="mean", softmax=True)

# Calcular pérdidas
loss_my = my_dice_loss(y_pred, y_true)
loss_monai = monai_dice_loss(y_pred, F.one_hot(y_true, num_classes=n_classes).permute(0, 3, 1, 2).float())

# Mostrar resultados
print(f"Dice Loss (Mi implementación): {loss_my.item():.6f}")
print(f"Dice Loss (MONAI): {loss_monai.item():.6f}")

Dice Loss (Mi implementación): 0.800123
Dice Loss (MONAI): 0.800124
